# Text Classification

In [1]:
#!pip install transformers datasets wandb

In [2]:
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,
                          DataCollatorWithPadding)
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### Example of Text Classification

Example: Sentiment Analysis

In [9]:
example_text = "I absolutely love this new phone! The battery life is amazing."

example_pipeline = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

output = example_pipeline(example_text)

print("\nText Classification Output:")
print(output)

Device set to use cuda:0



Text Classification Output:
[{'label': 'POSITIVE', 'score': 0.9998786449432373}]


Example: Zero Shot Classification

In [10]:
example_text = "The team secured a last-minute victory in the championship game."

example_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["technology", "sports", "politics"]

result = example_pipeline(example_text, candidate_labels)

max_index = result["scores"].index(max(result["scores"]))
best_label = result["labels"][max_index]

print("\nFull Output:", result)
print(f"\nBest Label: {best_label} (Score: {result['scores'][max_index]:.4f})")

Device set to use cuda:0



Full Output: {'sequence': 'The team secured a last-minute victory in the championship game.', 'labels': ['sports', 'technology', 'politics'], 'scores': [0.9376156330108643, 0.04836241528391838, 0.01402195356786251]}

Best Label: sports (Score: 0.9376)


### Full Text Classification Workflow

1- Dataset Preparation

In [26]:
dataset = load_dataset("imdb")

print("\nExample Review:", dataset["train"][0])


Example Review: {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes a

2- Tokenizer Initialization

In [27]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

3- Data Preprocessing

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

4- Model Loading

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5- Data Collation & Training Configuration

In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./text_classification_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
)

6- Evaluation Metrics

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

7- Model Training & Evaluation

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sauls2001 (sauls2001-universidade-da-coru-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.291600,0.324826,0.893120,0.898618,0.893120,0.892750
2,0.181300,0.373800,0.911920,0.911989,0.911920,0.911916
3,0.100100,0.410355,0.912400,0.912436,0.912400,0.912398


TrainOutput(global_step=9375, training_loss=0.21032844278971355, metrics={'train_runtime': 2443.5667, 'train_samples_per_second': 30.693, 'train_steps_per_second': 3.837, 'total_flos': 4967527449600000.0, 'train_loss': 0.21032844278971355, 'epoch': 3.0})

8- Model Saving & Inference

In [25]:
trainer.save_model("./fine_tuned_text_classifier")
tokenizer.save_pretrained("./fine_tuned_text_classifier")

text_classifier = pipeline("text-classification", model="./fine_tuned_text_classifier")

id2label = {"LABEL_0": "Negative", "LABEL_1": "Positive"}

example_text = "This was the worst movie I have ever seen."
output = text_classifier(example_text)[0]

output["label"] = id2label[output["label"]]

print("\nFinal Inference Output:")
print(output)

Device set to use cuda:0



Final Inference Output:
{'label': 'Negative', 'score': 0.9972202777862549}
